In [0]:
from pyspark.sql import functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import (
    LogisticRegression,
    DecisionTreeClassifier,
    RandomForestClassifier
)
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator


import mlflow
import mlflow.spark

In [0]:
df = spark.table("workspace.ecommerce.ecommerce_events_delta")

In [0]:
df = df.withColumn(
    "label",
    F.when(F.col("event_type") == "purchase", 1).otherwise(0)
)

In [0]:

feature_cols = ["price", "product_id", "category_id"]

train_df, test_df = (
    df
    .select(feature_cols + ["label"])
    .randomSplit([0.8, 0.2], seed=42)
)


assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)
     

In [0]:
models = {
    "logistic_regression": LogisticRegression(),
    "decision_tree": DecisionTreeClassifier(maxDepth=5),
    "random_forest": RandomForestClassifier(numTrees=50)
}
   

In [0]:
evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    metricName="areaUnderROC"
)
   

In [0]:
accuracy_evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    metricName="accuracy"
)

In [0]:
import os

os.environ["MLFLOW_DFS_TMP"] = "/Volumes/workspace/ecommerce/ecommerce_data/mlflow_tmp"


In [0]:
results = []

for name, model in models.items():

    pipeline = Pipeline(stages=[assembler, model])

    with mlflow.start_run(run_name=name, nested=True):

        mlflow.log_param("model_name", name)

        trained_model = pipeline.fit(train_df)
        predictions = trained_model.transform(test_df)

        auc = evaluator.evaluate(predictions)
        accuracy = accuracy_evaluator.evaluate(predictions)

        mlflow.log_metric("AUC", auc)
        mlflow.log_metric("Accuracy", accuracy)

        mlflow.spark.log_model(
            trained_model,
            artifact_path="model",
            dfs_tmpdir="/Volumes/workspace/ecommerce/ecommerce_data/mlflow_tmp"
        )

        results.append((name, auc, accuracy))
        print(f"{name} → AUC = {auc:.4f}, Accuracy = {accuracy:.4f}")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6426400283418349>, line 11
      7 with mlflow.start_run(run_name=name, nested=True):
      9     mlflow.log_param("model_name", name)
---> 11     trained_model = pipeline.fit(train_df)
     12     predictions = trained_model.transform(test_df)
     14     auc = evaluator.evaluate(predictions)

File /databricks/python_shell/lib/dbruntime/MLWorkloadsInstrumentation/_pyspark.py:30, in _create_patch_function.<locals>.patched_method(self, *args, **kwargs)
     28 call_succeeded = False
     29 try:
---> 30     result = original_method(self, *args, **kwargs)
     31     call_succeeded = True
     32     return result

File /databricks/python/lib/python3.12/site-packages/pyspark/ml/base.py:203, in Estimator.fit(self, dataset, params)
    201         return self.copy(params)._fit(dataset)
    202     else:
--> 203         return 

In [0]:
best_model = max(results, key=lambda x: x[1])  # based on AUC

print("\n🏆 Best Model Selected:")
print(f"Model Name : {best_model[0]}")
print(f"AUC        : {best_model[1]:.4f}")
print(f"Accuracy   : {best_model[2]:.4f}")